In [21]:
import pandas as pd 
import geopandas as gpd
import fiona

In [27]:
from zipfile import ZipFile
kmz = ZipFile('transit_stations/BART_System_2020.kmz', 'r')
kmz.extract('doc.kml', 'bart')

'bart/doc.kml'

In [52]:
#Caltrain Stations
caltrain = gpd.read_file('transit_stations/Caltrain Stations and Stops/geo_export_c83debd0-dfd7-4ffb-88ad-a03a92dfdcf9.shp')

#Bart Stations 
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
bart = gpd.read_file('bart/doc.kml', driver='KML', crs = 'EPSG:4326')

In [43]:
filter_1 = caltrain.stop_code != 0.0 #Not stations 
filter_2 = caltrain.platform_c == 'NB' #Only NB or SB - Duplicated otherwise
filter_3 = caltrain.stop_name != 'Atherton Caltrain' #Station Closed in Dec 2020

df_caltrain = caltrain[filter_1 & filter_2  & filter_3 ]

In [68]:
caltrain['agency'] = 'caltrain'
c = caltrain[['stop_name', 'geometry', 'agency']]

In [69]:
bart['agency'] = 'bart'
b = bart[['Name', 'geometry','agency']].rename(columns = {'Name':'stop_name'})

In [72]:
stations = pd.concat((b,c))

In [76]:
stations['x'] = stations.geometry.x
stations['y'] = stations.geometry.y

In [80]:
stations.geometry = gpd.points_from_xy(stations.x, stations.y, crs = 'EPSG:4326')

In [86]:
stations.to_file('mass_transit_stations_bay_area.shp')